# Descarga de datos

In [1]:
from google.colab import output
!pip install gdown
!gdown https://drive.google.com/uc?id=1L5kNJEwRRlNtBvXVt4KZB0EiTQk23iaV # pokedex_processed.csv
!gdown https://drive.google.com/uc?id=19dw2vVgF6pht_-GF5_IvxrVLOSkT8kCQ # pokedex_processed.json
output.clear()

# Datos

Los datos corresponden al mundo de _Pokémon_ separado en dos bases de datos: Una relacional SQL (PostgreSQL) y otra de documentos NoSQL (MongoDB). Ambas son levantadas dentro del entorno de ejecución del notebook en Google Colab.

A continuación se crea y describe a cada una:

# PostgreSQL

Cada columna de la tabla de pokedex se describe a continuación:

```text
###################
## Tabla pokedex ##
###################

Id           : (int) Identificador único en el Pokédex
Name         : (str) Nombre del Pokémon
Category     : (str) La "especie" del Pokémon (en Pokémon Sol y Luna)
Height (m)   : (int) Altura en metros
Weight (kg)  : (int) Masa en kilogramos
Capture Rate : (int) Cuantifica la probabilidad de capturar al Pokémon (a mayor valor, más probable).
Egg Steps    : (int) Cuantifica el número de pasos que requiere en el juego para salir de un huevo.
Exp Group    : (str) Categoría que determina cuanta experiencia necesita para subir de nivel.
HP           : (int) Vida
Attack       : (int) Ataque
Defense      : (int) Defensa
Sp. Attack   : (int) Ataque Especial
Sp. Defense  : (int) Defensa Especial
Speed        : (int) Velocidad

```

In [2]:
# install
!apt update
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# pin package versions for google colab compatibility
!pip install SQLAlchemy==1.4.46
!pip install ipython-sql==0.4.1
# set connection
%load_ext sql
%config SqlMagic.feedback=False
%config SqlMagic.autopandas=True
%config SqlMagic.displaycon = False
%sql postgresql+psycopg2://@/postgres
output.clear()

In [3]:
import pandas as pd
from sqlalchemy import create_engine
engine = create_engine("postgresql+psycopg2://@/postgres", echo=False)
df_pokedex = pd.read_csv("pokedex_processed.csv", index_col=False)
df_pokedex.to_sql("pokedex", con=engine, if_exists="replace", index=False)

809

In [4]:
%%sql
SELECT *
FROM pokedex
LIMIT 10

,Id,Name,Category,Height (m),Weight (kg),Capture Rate,Egg Steps,Exp Group,HP,Attack,Defense,Sp. Attack,Sp. Defense,Speed
0,1,Bulbasaur,Seed Pokémon,0.7,6.9,45,5120,Medium Slow,45,49,49,65,65,45
1,2,Ivysaur,Seed Pokémon,1.0,13.0,45,5120,Medium Slow,60,62,63,80,80,60
2,3,Venusaur,Seed Pokémon,2.0,100.0,45,5120,Medium Slow,80,82,83,100,100,80
3,4,Charmander,Lizard Pokémon,0.6,8.5,45,5120,Medium Slow,39,52,43,60,50,65
4,5,Charmeleon,Flame Pokémon,1.1,19.0,45,5120,Medium Slow,58,64,58,80,65,80
5,6,Charizard,Flame Pokémon,1.7,90.5,45,5120,Medium Slow,78,84,78,109,85,100
6,7,Squirtle,Tiny Turtle Pokémon,0.5,9.0,45,5120,Medium Slow,44,48,65,50,64,43
7,8,Wartortle,Turtle Pokémon,1.0,22.5,45,5120,Medium Slow,59,63,80,65,80,58
8,9,Blastoise,Shellfish Pokémon,1.6,85.5,45,5120,Medium Slow,79,83,100,85,105,78
9,10,Caterpie,Worm Pokémon,0.3,2.9,255,3840,Medium Fast,45,30,35,20,20,45


## MongoDB

La colección pokedex tiene documentos con 4 entradas: Un identificador único del pokémon y arreglos con información acerca del mismo. A continuación se detalla cada uno:

```text
#######################
## Colección pokedex ##
#######################

"pokemon_id": : (int)       Número identificador único en el Pokédex
"abilities"   : (List[str]) Lista de las habilidades del Pokémon
"moves"       : (List[str]) Lista de los movimientos del Pokémon
"types"       : (List[str]) Lista de los tipos del Pokémon
```

In [5]:
!apt install mongodb
!service mongodb start
!pip install pymongo
!mongoimport --db local --collection pokedex --jsonArray < 'pokedex_processed.json'
output.clear()

In [6]:
from pymongo import MongoClient
client = MongoClient()
client.list_database_names() # ['admin', 'local']
db = client['local']
collection = db['pokedex']

Para tener una idea de cómo se ven los datos en la colección, aquí les dejamos impresión del primer Pokemon en la colección:

In [7]:
import pprint
for pokemon in collection.find({}).limit(1):
    pprint.pprint(pokemon)

{'_id': ObjectId('648b5d8bac4a55339417f047'),
 'abilities': ['Overgrow', 'Chlorophyll'],
 'moves': ['Tackle is described as A physical attack in which the user charges '
           'and slams into the target with its whole body.',
           'Growl is described as The user growls in an endearing way, making '
           'opposing Pokémon less wary. This lowers their Attack stat.',
           'Vine Whip is described as The target is struck with slender, '
           'whiplike vines to inflict damage.',
           'Poison Powder is described as The user scatters a cloud of '
           'poisonous dust that poisons the target.',
           'Sleep Powder is described as The user scatters a big cloud of '
           'sleep-inducing dust around the target.',
           'Take Down is described as A reckless, full-body charge attack for '
           'slamming into the target. This also damages the user a little.',
           'Razor Leaf is described as Sharp-edged leaves are launched to '
    

# Preguntas

Para resolver estas preguntas será necesario que consultes a `PostgreSQL` y/o `MongoDB`.

En este `notebook` puedes almacenar el resultado de una consulta `PostgreSQL` en una variable como un `pandas.DataFrame` de la siguiente forma:

```python
df = %sql SELECT "Name" FROM pokedex WHERE "Id" = 99;
```

## Pregunta 1
Entregue el nombre (name) y la cantidad de habilidades (abilites) de los 10 pokemones más pesados en kilogramos

In [8]:
Pokedex =  %sql SELECT * FROM pokedex;

In [9]:
project_stage = {'$project': {'_id': 0, 'pokemon_id': 1, 'abilities': 1}}

# Etapa $addFields para agregar un campo 'num_moves' con la cantidad de movimientos
add_fields_stage = {'$addFields': {'num_abilities': {'$size': '$abilities'}}}

# Ejecutar la consulta agregada
result = collection.aggregate([project_stage, add_fields_stage])

habilidades ={}
# Imprimir los resultados
for doc in result:
  habilidades[doc['pokemon_id']] = doc['num_abilities']
sorted_habilidades = dict(sorted(habilidades.items()))

In [10]:
Pokedex['Num_Abilities'] = Pokedex['Id'].map(sorted_habilidades)
respuesta_1 = Pokedex.sort_values('Weight (kg)')[['Name', 'Num_Abilities']].head(10)

In [11]:
respuesta_1

,Name,Num_Abilities
788,Cosmog,1
92,Haunter,1
91,Gastly,1
797,Kartana,1
668,Flabébé,2
741,Cutiefly,3
601,Tynamo,1
745,Wishiwashi,1
481,Azelf,1
479,Uxie,1


## Pregunta 2

Nombre los 3 pokemones con más movimientos. Se recomienda revisar la documentación de MongoDB, particularmente de [`$sort`](https://www.mongodb.com/docs/manual/reference/operator/aggregation/sort/) y [`$limit`](https://www.mongodb.com/docs/manual/reference/operator/aggregation/limit/).

In [12]:
from itertools import islice
# Etapa $project para extraer solo los campos necesarios ('_id' y 'moves')
project_stage = {'$project': {'_id': 0, 'pokemon_id': 1, 'moves': 1}}

# Etapa $addFields para agregar un campo 'num_moves' con la cantidad de movimientos
add_fields_stage = {'$addFields': {'num_moves': {'$size': '$moves'}}}

# Etapa $sort para ordenar los documentos por el campo 'num_moves' en orden descendente
sort_stage = {'$sort': {'num_moves': -1}}

# Ejecutar la consulta agregada
result = collection.aggregate([project_stage, add_fields_stage, sort_stage])

movimientos = {}
# Imprimir los resultados
for doc in result:
    movimientos[doc['pokemon_id']] = doc['num_moves']
list(movimientos.keys())[0:3]
new_movimientos = dict(islice(movimientos.items(), 3))

Pokedex['Num_Moves'] = Pokedex['Id'].map(new_movimientos)
respuesta_2 = Pokedex[Pokedex['Num_Moves'].notnull()][['Name', 'Num_Moves']].sort_values('Num_Moves', ascending=False)

In [13]:
respuesta_2

,Name,Num_Moves
772,Silvally,26.0
207,Steelix,25.0
121,Mr. Mime,24.0


## Pregunta 3

Cuál es el elemento de los pokemones más densos? Indique los 5 elementos de pokemones más densos. Asuma que la densidad es el peso dividido en la altura, y que los pokemones tienen un solo tipo (ignore todos los elementos de cada pokemon excepto uno).

In [14]:
Density = Pokedex['Weight (kg)']/Pokedex['Height (m)']
Pokedex['Density'] = Density

# Etapa $project para extraer solo los campos necesarios ('_id' y 'moves')
project_stage = {'$project': {'_id': 0, 'pokemon_id': 1, 'types': 1}}

Elementos = {}
for pokemon in collection.find({}):
  Elementos[pokemon['pokemon_id']] = pokemon['types'][0]

In [15]:
Pokedex['Elemento'] = Pokedex['Id'].map(Elementos)

In [16]:
respuesta_3 = Pokedex.sort_values('Density')[['Name', 'Elemento']].head(5)

In [17]:
respuesta_3

,Name,Elemento
92,Haunter,Ghost
91,Gastly,Ghost
797,Kartana,Grass
788,Cosmog,Psychic
481,Azelf,Psychic


## Pregunta 4

Vea si existe alguna correlación entre algún atributo (ataque, defensa, ataque especial, defensa especial, velocidad y HP) y la cantidad de movimientos que tienen los pokemones.

Hay algun otro par de variables (de las mencionadas anteriormente) que tengan mayor correlación entre sí? Indique el par de atributos que tengan la mayor correlación entre sí y haga un gráfico de dispersión para visualizar la correlación.

In [18]:
Pokedex['Num_Moves'] = Pokedex['Id'].map(movimientos)
Pokedex[['Attack', 'Sp. Attack', 'Defense', 'Sp. Defense', 'Speed', 'HP', 'Num_Moves']]

,Attack,Sp. Attack,Defense,Sp. Defense,Speed,HP,Num_Moves
0,49,65,49,65,45,45,13
1,62,80,63,80,60,60,14
2,82,100,83,100,80,80,16
3,52,60,43,50,65,39,12
4,64,80,58,65,80,58,12
...,...,...,...,...,...,...,...
804,131,53,211,101,13,61,13
805,127,151,53,79,107,53,12
806,112,102,75,80,143,88,16
807,65,55,65,35,34,46,7


In [19]:
#Elegí attack.
correlacion = Pokedex['Attack'].corr(Pokedex['Num_Moves'])
print(f'La correlación entre el ataque y el numero de movimientos {correlacion}')

La correlación entre el ataque y el numero de movimientos 0.2536648735278


In [20]:
Pokedex[['Attack', 'Sp. Attack', 'Defense', 'Sp. Defense', 'Speed', 'HP', 'Num_Moves']].corr()

,Attack,Sp. Attack,Defense,Sp. Defense,Speed,HP,Num_Moves
Attack,1.000000,0.326032,0.439335,0.201357,0.330985,0.437478,0.253665
Sp. Attack,0.326032,1.000000,0.199997,0.481435,0.439785,0.377330,0.181285
Defense,0.439335,0.199997,1.000000,0.506163,-0.026810,0.237430,0.212476
Sp. Defense,0.201357,0.481435,0.506163,1.000000,0.204384,0.366738,0.226473
Speed,0.330985,0.439785,-0.026810,0.204384,1.000000,0.166122,0.125093
HP,0.437478,0.377330,0.237430,0.366738,0.166122,1.000000,0.180650
Num_Moves,0.253665,0.181285,0.212476,0.226473,0.125093,0.180650,1.000000


Los atributos con mayor correlación son Defensa y Defensa especial.

## Importante:
Para las últimas preguntas será necesario que crees y consultes un índice de texto para el arreglo de strings `moves`.

In [21]:
# Aquí escribe el código para crear el índice de texto
collection.create_index([('moves', 'text')])

'moves_text'

## Pregunta 5
Entregue el nombre de todos los pokemones los cuales en sus movimientos contengan las palabras `fire` y `thunder` y tengan un Ataque Especial (Sp. Attack) entre `110` y `130` (ambos valores incluidos)

In [22]:
Indexes = set()
#{'Column1': {'$in': value_list}}
for pokemon in collection.find({'$text': {'$search': '\"fire\" \"thunder\"'}}):
    Indexes.add(pokemon['pokemon_id'])

In [23]:
Filtered_Pokedex = Pokedex[Pokedex['Id'].isin(Indexes)]
respuesta_5 = Filtered_Pokedex[(Filtered_Pokedex['Sp. Attack'] >= 110) & (Filtered_Pokedex['Sp. Attack'] <= 130)]['Name']

In [24]:
print(f'Nombres validos: \n{respuesta_5}')

Nombres validos: 
81      Magneton
144       Zapdos
180     Ampharos
228     Houndoom
242       Raikou
372    Salamence
461    Magnezone
466    Magmortar
Name: Name, dtype: object


## Pregunta 6
Entregue el nombre junto con la suma de las estadísticas de combate (HP, Attack, Defense, Sp. Attack, Sp. Defense y Speed) de los pokemones que en sus movimientos contengan las palabras `ice`, `freeze`, `snow` o `cold`, **pero no** las palabras `water` y `wind`. Deben estar ordenado alfabéticamente por su nombre.

In [25]:
Estadisticas = Pokedex[['Attack', 'Sp. Attack', 'Defense', 'Sp. Defense', 'Speed', 'HP']].sum(axis=1)
Pokedex['Estadistics'] = Estadisticas

In [26]:
Indexes_2 = set()
for pokemon in collection.find({'$text': {'$search': 'ice, freeze, snow, cold -water -wind'}}):
    Indexes_2.add(pokemon['pokemon_id'])
Filtered_Pokedex = Pokedex[Pokedex['Id'].isin(Indexes_2)]

In [27]:
respuesta_6 = Filtered_Pokedex[['Name', 'Estadistics']].sort_values('Name')

In [28]:
respuesta_6

,Name,Estadistics
141,Aerodactyl,515
23,Arbok,448
793,Buzzwole,570
317,Carvanha,305
739,Crabominable,478
84,Dodrio,470
451,Drapion,500
50,Dugtrio,425
355,Dusclops,455
476,Dusknoir,525
